In [ ]:
# default_exp Starting-off

# Looking around

> We'll try and get a broad overview of nn.Module in this section

In [ ]:
#hide
%reload_ext autoreload
%autoreload 2
%config autocompleter.use_jedi=False
from nbdev.showdoc import *

In [ ]:
from fastai.vision import *
import torch.nn as nn
import torch.nn.functional as F
import torch
import numpy as np
import matplotlib.pyplot as plt
randn = torch.randn

In [ ]:
#export
def print_all(*args, **kwargs):
    """ Prints all parameters on separate lines """
    for i in args:
        print(i)
    for key, value in kwargs.items():
        print(f"{key} : {value}")

In [ ]:
print(', '.join(dir(nn.Module)))

__call__, __class__, __delattr__, __dict__, __dir__, __doc__, __eq__, __format__, __ge__, __getattr__, __getattribute__, __gt__, __hash__, __init__, __init_subclass__, __le__, __lt__, __module__, __ne__, __new__, __reduce__, __reduce_ex__, __repr__, __setattr__, __setstate__, __sizeof__, __str__, __subclasshook__, __weakref__, _apply, _get_name, _load_from_state_dict, _named_members, _register_load_state_dict_pre_hook, _register_state_dict_hook, _slow_forward, _tracing_name, _version, add_module, apply, buffers, children, cpu, cuda, double, dump_patches, eval, extra_repr, float, forward, half, load_state_dict, modules, named_buffers, named_children, named_modules, named_parameters, parameters, register_backward_hook, register_buffer, register_forward_hook, register_forward_pre_hook, register_parameter, share_memory, state_dict, to, train, type, zero_grad


We will try cover as many important parts of the `nn.Module` class as possible.


##### nn.Module's documentaions says:

---
CLASS `torch.nn.Module`:

*Base class for all neural network modules.*

*Your models should also subclass this class.*

*Modules can also contain other Modules, allowing to nest them in a tree structure. You can assign the submodules as regular attributes:*
```python
import torch.nn as nn
import torch.nn.functional as F

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5)
        self.conv2 = nn.Conv2d(20, 20, 5)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        return F.relu(self.conv2(x))
```
*Submodules assigned in this way will be registered, and will have their parameters converted too when you call `to()`, etc.*

Things to note:
1. Modules can have submodules
2. Every module is **registered**. We'll explore what this means soon.
3. The method `to()`. We will look into this too.

# Creating a basic CNN network

In [ ]:
class Model_1(nn.Module):
    """ A basic CNN Model with Adaptive Pooling to allow varying inpup sizes """
    def __init__(self):
        super(Model_1, self).__init__()
        self.conv1 = nn.Conv2d(3, 4, 3)
        self.conv2 = nn.Conv2d(4, 16, 3)
        self.conv3 = nn.Conv2d(16, 4, 3)
        self.pool1 = nn.AdaptiveAvgPool2d(1)
        self.lin1  = nn.Linear(4, 2)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = self.pool1(x)
        x = x.view(x.shape[0], -1) # Flattening X
        return self.lin1(x)

net = Model_1()

In [ ]:
#export
def test_net(net, x=randn(4, 3, 100, 100)):
    """ A convenience function to test out all the various models that we will be creating """
    out = net(x)
    print_all(out.shape, out)

In [ ]:
test_net(net)

torch.Size([4, 2])
tensor([[0.2610, 0.0375],
        [0.2608, 0.0372],
        [0.2610, 0.0374],
        [0.2610, 0.0372]], grad_fn=<AddmmBackward>)


---
# Our Goals Today:
1. We will try and create the above network in as many ways as possible using the functions available to us and we will go on slight tangents along the way as we find more interesting things.
2. We will look into the source code for nn.Module to gain more insight into it's internals.
3. We will also see how some of the convenience functions provided in the module can help us.

# Goal 1: Recreating `Model_1`

All the possible ways:
```python
1. nn.ModuleList
2. nn.ModuleDict
3. add_module
4. nn.ParameterList
5. nn.ParameterDict
6. register_parameter
```

## `nn.Sequential` : The easiest alterenative
> Not necessarily the most convenient

In [ ]:
#export
class LambdaModule(nn.Module):
    def __init__(self, func):
        super(LambdaModule, self).__init__()
        self.func = func
    def forward(self, x):
        return self.func(x)

In [ ]:
Flatten = LambdaModule(lambda x: x.view(x.shape[0], -1))
Flatten(randn(3,4,4)).shape # Testing Flatten

torch.Size([3, 16])

In [ ]:
model = nn.Sequential(nn.Conv2d(3, 4, 3),
                      nn.ReLU(),
                      nn.Conv2d(4, 16, 3),
                      nn.ReLU(),
                      nn.Conv2d(16, 4, 3),
                      nn.ReLU(),
                      nn.AdaptiveAvgPool2d(1),
                      Flatten,
                      nn.Linear(4, 2)
                     )

In [ ]:
test_net(model)

torch.Size([4, 2])
tensor([[0.4961, 0.4088],
        [0.4950, 0.4097],
        [0.4956, 0.4092],
        [0.4945, 0.4100]], grad_fn=<AddmmBackward>)


In [ ]:
# We will need this a lot going ahead
def get_module_list():
    modules = [nn.Conv2d(3, 4, 3),
               nn.ReLU(),
               nn.Conv2d(4, 16, 3),
               nn.ReLU(),
               nn.Conv2d(16, 4, 3),
               nn.ReLU(),
               nn.AdaptiveAvgPool2d(1),
               Flatten,
               nn.Linear(4, 2)]
    return modules

## `nn.ModuleList`

*Provide description here of how it works*

In [ ]:
class Model(nn.Module):
    def __init__(self, layers:list):
        super(Model, self).__init__()
        self.layers = nn.ModuleList(layers)

    def forward(self, x):
        # ModuleList can act as an iterable, or be indexed using ints
        for l in self.layers:
            x = l(x)
        return x

In [ ]:
net = Model(get_module_list())

In [ ]:
test_net(net)

torch.Size([4, 2])
tensor([[ 0.1547, -0.3233],
        [ 0.1546, -0.3233],
        [ 0.1551, -0.3236],
        [ 0.1553, -0.3235]], grad_fn=<AddmmBackward>)
